## Build BioMistral Medical RAG Chatbot using BioMistral Open Source LLM

In [3]:
!pip install langchain sentence-transformers chromadb llama-cpp-python langchain_community pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 MB 12.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━

## Load the Drive

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


### Importing libraries

In [4]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain_community.llms import LlamaCpp
from langchain.chains import RetrievalQA, LLMChain

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


## Import the document

In [5]:
loader = PyPDFDirectoryLoader("/content/data")
docs = loader.load()

In [6]:
print(len(docs))

95


In [7]:
docs[3]

Document(metadata={'source': '/content/data/healthyheart.pdf', 'page': 3}, page_content='Heart Disease: Why Should You Care? . . . . . . . . . . . . . . . . . . . . . . . . . . . . 1\nWhat You Need To Know About Heart Disease . . . . . . . . . . . . . . . . . . . . . . 3\nWhat Is Heart Disease?. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 3Who Is at Risk? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 4How Risk Works . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 4\nWhat’s Your Risk? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5\nHow To Talk With Your Doctor . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 5\nMajor Risk Factors. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 11\nSmoking . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 

## Chunking

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 300, chunk_overlap = 50)
chunks = text_splitter.split_documents(docs)


In [9]:
len(chunks)

747

In [10]:
chunks[34]

Document(metadata={'source': '/content/data/healthyheart.pdf', 'page': 4}, page_content='You \nCanStop Smoking. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 76')

## Embedding creations

In [11]:
import os

In [12]:
os.environ['HUGGINGFACEHUB_API_TOKEN'] = "use_your_huggingface_key"

In [13]:
embeddings = SentenceTransformerEmbeddings(model_name = "NeuML/pubmedbert-base-embeddings")

<ipython-input-13-85d430a8eb2d>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name = "NeuML/pubmedbert-base-embeddings")
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/706k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/74.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Vector Store creation

In [14]:
vectorstore = Chroma.from_documents(chunks, embeddings)

In [15]:
query = "Who is at risk of heart disease?"

search_results = vectorstore.similarity_search(query)

In [16]:
search_results

[Document(metadata={'page': 8, 'source': '/content/data/healthyheart.pdf'}, page_content='While each risk factor increases your risk of heart disease, having'),
 Document(metadata={'page': 5, 'source': '/content/data/healthyheart.pdf'}, page_content='■As early as age 45, a man’s risk of heart disease begins to rise significantly. For a woman, risk starts to increase at age 55.\n■Fifty percent of men and 64 percent of women who die suddenlyof heart disease have no previous symptoms of the disease.1Heart Disease: Why Should You Care?Heart Disease:'),
 Document(metadata={'page': 6, 'source': '/content/data/healthyheart.pdf'}, page_content='factor greatly increases the chances of developing heart disease.Moreover, the worse a particular risk factor is, the more likely youare to develop heart disease. For example, if you have high bloodpressure, the higher it is, the greater your chances of developingheart disease, including its many'),
 Document(metadata={'page': 24, 'source': '/content/da

In [17]:
retriever = vectorstore.as_retriever(search_kwargs = {'k':5})

In [18]:
retriever.get_relevant_documents(query)

<ipython-input-18-162c3489abc9>:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents(query)


[Document(metadata={'page': 8, 'source': '/content/data/healthyheart.pdf'}, page_content='While each risk factor increases your risk of heart disease, having'),
 Document(metadata={'page': 5, 'source': '/content/data/healthyheart.pdf'}, page_content='■As early as age 45, a man’s risk of heart disease begins to rise significantly. For a woman, risk starts to increase at age 55.\n■Fifty percent of men and 64 percent of women who die suddenlyof heart disease have no previous symptoms of the disease.1Heart Disease: Why Should You Care?Heart Disease:'),
 Document(metadata={'page': 6, 'source': '/content/data/healthyheart.pdf'}, page_content='factor greatly increases the chances of developing heart disease.Moreover, the worse a particular risk factor is, the more likely youare to develop heart disease. For example, if you have high bloodpressure, the higher it is, the greater your chances of developingheart disease, including its many'),
 Document(metadata={'page': 24, 'source': '/content/da

## LLM Model Loading

In [19]:
llm = LlamaCpp(
    model_path = "/content/drive/MyDrive/BioMistral-7B.Q4_K_M.gguf",
    temparature = 0.2,
    max_tokens = 2048,
    top_p = 1
)

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3473: UserWarning: WARNING! temparature is not default parameter.
                temparature was transferred to model_kwargs.
                Please confirm that temparature is what you intended.
  if (await self.run_code(code, result,  async_=asy)):
llama_model_loader: loaded meta data with 21 key-value pairs and 291 tensors from /content/drive/MyDrive/BioMistral-7B.Q4_K_M.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = hub
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4: 

## Use LLM and Retriever and Query, to generate final response

In [20]:
template = """
<|context|>
You are an Medical Assisstant that follows the instructions and generate the accurate response based on the query and the context provided
Please be truthful and give direct answers.
</s>
<|user|>
{query}
</s>
<|assisstant|>
"""

In [21]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.prompts import ChatPromptTemplate


In [22]:
prompt = ChatPromptTemplate.from_template(template)

In [23]:
rag_chain = (
    {"context": retriever, "query": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [24]:
response = rag_chain.invoke(query)


llama_print_timings:        load time =    2687.66 ms
llama_print_timings:      sample time =      32.80 ms /    56 runs   (    0.59 ms per token,  1707.21 tokens per second)
llama_print_timings: prompt eval time =   29200.27 ms /    75 tokens (  389.34 ms per token,     2.57 tokens per second)
llama_print_timings:        eval time =   36425.35 ms /    55 runs   (  662.28 ms per token,     1.51 tokens per second)
llama_print_timings:       total time =   65717.35 ms /   130 tokens


In [25]:
response

'The risk factors of heart disease include high blood pressure, diabetes, obesity and smoking. If you have any of these conditions, you are at a higher risk for developing heart disease. However, it’s never too late to make lifestyle changes and reduce your risks.'

In [26]:
import sys

while True:
  user_input = input(f"Input Query: ")
  if user_input == 'exit':
    print("Exiting......")
    sys.exit()
  if user_input =="":
    continue
  result = rag_chain.invoke(user_input)
  print("Answer: ", result)

Input Query: What are the diseases that affect heart health?


Llama.generate: 55 prefix-match hit, remaining 21 prompt tokens to eval

llama_print_timings:        load time =    2687.66 ms
llama_print_timings:      sample time =      39.61 ms /    60 runs   (    0.66 ms per token,  1514.77 tokens per second)
llama_print_timings: prompt eval time =    6884.14 ms /    21 tokens (  327.82 ms per token,     3.05 tokens per second)
llama_print_timings:        eval time =   35829.19 ms /    59 runs   (  607.27 ms per token,     1.65 tokens per second)
llama_print_timings:       total time =   42803.87 ms /    80 tokens


Answer:  There are many heart disorders and conditions, including but not limited to: • Cardiomyopathies • Arrhythmias • Congenital Heart Defects • Coronary Heart Disease • Endocarditis • Hypertension • Myocarditis • Valvular Diseases
Input Query: How high Blood cholesterol affect heart health?


Llama.generate: 55 prefix-match hit, remaining 23 prompt tokens to eval

llama_print_timings:        load time =    2687.66 ms
llama_print_timings:      sample time =      42.14 ms /    74 runs   (    0.57 ms per token,  1756.09 tokens per second)
llama_print_timings: prompt eval time =    7359.59 ms /    23 tokens (  319.98 ms per token,     3.13 tokens per second)
llama_print_timings:        eval time =   41988.76 ms /    73 runs   (  575.19 ms per token,     1.74 tokens per second)
llama_print_timings:       total time =   49448.32 ms /    96 tokens


Answer:  High blood cholesterol levels can lead to plaque buildup in your arteries, increasing the risk of heart disease such as coronary artery disease. High blood cholesterol levels are also associated with an increased risk of heart attack and stroke. Maintaining healthy blood cholesterol levels is important for overall cardiovascular health.
Input Query: exit
Exiting......


SystemExit: 

/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
